In [ ]:
%matplotlib notebook
import os
from adm_library import *

## 0 Parameter definitions

### 0.1 Ingesting the [X,Y,Z,Rx,Ry] values from Tyler's spreadsheet

In [ ]:
fname = 'files/SES-cold3-C4- 5DoF inputs_template.xlsx'
sheetname = 'Input for Tyler analysis'
#fname = 'SES-cold plateau 3 - circuit 2.xlsx'
#sheetname = 'Input for Tyler analysis'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:9,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]
pose_encoders = spreadsheet.iloc[0:9,7:13]
pose_encoders = pose_encoders.set_index('Name.1').sort_index()

print('Provided 5DOF position in real space: \n',pose_actual,'\n')
print('Provided 5DOF encoder values: \n',pose_encoders)


### 0.1.1 If both the pose_actual and pose_encoder arrays are provided in the spreadsheet above, convert between the two to make sure they both agree. 

In [ ]:
#No point doing the following if one or the other array is completely empty
if (not any(np.isfinite(pose_encoders.values.astype(float)).ravel())) and (not any(np.isfinite(pose_actual.values.astype(float)).ravel())):

    print('(Calculated 5DOF position) - (Actual 5DOF position (from the spreadsheet)): ')
    calculated_pose_actual = pd.DataFrame(columns=pose_actual.columns)
    for index in [val for val in pose_actual.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_actual = pd.concat((calculated_pose_actual,calculate_5DOF_from_encoders(pd.DataFrame(pose_encoders.loc[index]).T)))
    print(calculated_pose_actual-pose_actual,'\n\n')
    
    print('(Calculated 5DOF encoders) - (Actual 5DOF encoders (from the spreadsheet)): ')
    calculated_pose_encoders = pd.DataFrame(columns=pose_encoders.columns)
    for index in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
        calculated_pose_encoders = pd.concat((calculated_pose_encoders,calculate_encoders_from_5DOF(pd.DataFrame(pose_actual.loc[index]).T)))
    print(calculated_pose_encoders-pose_encoders)

### 0.2 If the "pose_actual" section of the supplied spreadsheet was empty, build a new "pose_actual" array based on the input "pose_encoder" array.

In [ ]:
if not any(np.isfinite(pose_actual.values.astype(float)).ravel()):
    pose_actual = convert_pose_encoders_to_pose_actual(pose_encoders)
    print('NEWLY calculated 5DOF position in real space based on recorded 5DOF encoder values: \n',pose_actual)

### 0.3 Form a dataframe with the values supplied so far

In [ ]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA',['Rx','Ry','Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

#ax = plot_poses(df)
#plot_sMPA(df, ax)
df

# 1 Updating poses based on results from FDPR

### 1.1 Reading in offset data from the FDPR team's spreadsheet.

In [157]:
#These data will be used to update the position of the calculated endpoints in the sMPA frame
shift_data = pd.read_excel('files/FDPR shifts - sandbox.xlsx',sheet_name='PSF Shifts',skiprows=0,usecols='C:K')
shift_data = shift_data.iloc[0:6]
shift_data.set_index('camera',inplace=True)
print(shift_data)

        det normal dz (um)  chief ray dz (um)    row  ...  dy (px)  dx (um)  dy (um)
camera                                                ...                           
PR1                     25                 27  244.0  ...     12.0     -117      120
PR3                     -1                  0  238.9  ...     17.1     -130      171
PR4                     37                 45  234.1  ...     21.9      -99      219
PR5                     -4                 -3  240.9  ...     15.1     -161      151
PR2                     62                 64  229.1  ...     26.9     -181      269

[5 rows x 8 columns]


#### 1.1.1 Calculate some transformation variables necessary for converting between the 5DOF and sMPA frame

In [158]:
translation_to_sMPA = df.loc['sMPA',['X','Y','Z']]
rotation_from_sMPA_to_5DOF = R.from_matrix(rotmat_from_2vec(np.array([0,0,1]),df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']].values.astype(float)))

## Scratch

### 1.2 Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 5DOF optimization)

In [164]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals (in sMPA frame) baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.117267 -0.114330  0.022707
PR2  0.169774 -0.262177  0.049962
PR3  0.130404 -0.155604  0.070167
PR4  0.107572 -0.203885  0.045909
PR5  0.161898 -0.137013  0.064832 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.1075 -0.2338  0.0622 -0.0027  0.0013] 

[ 0.10751205 -0.2338194   0.06218    -0.00271954  0.00134254]
Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.021    0.059   -0.028
PR2    0.033   -0.085    0.010
PR3   -0.007    0.019    0.019
PR4   -0.030   -0.031   -0.016
PR5    0.025    0.037    0.014 

endpt_X    0.024554
endpt_Y    0.051735
endpt_Z    0.018308
dtype: float64


### 1.2 (alternate) Calculate the new best-fit poses that minimize the deltas observed by the FDPR team (with 6DOF optimization)

In [165]:
df_after_fitting_FDPR_shifts,endpoints_residuals_sMPA,best_fit_deltas = pose_update_with_FDPR_results_v2(df,shift_data,focal_length,GSA_angle_WCS_deg,translation_to_sMPA,rotation_from_sMPA_to_5DOF)
print(np.std(endpoints_residuals_sMPA))

Residuals (in sMPA frame) baseline: 
       endpt_X   endpt_Y   endpt_Z
PR1  0.117267 -0.114330  0.022707
PR2  0.169774 -0.262177  0.049962
PR3  0.130404 -0.155604  0.070167
PR4  0.107572 -0.203885  0.045909
PR5  0.161898 -0.137013  0.064832 

Best-fit (X,Y,Z,Rx,Ry) deltas to apply to real-space poses: 
 [ 0.1076 -0.2336  0.0621 -0.0027  0.0008 -0.0026] 

[ 0.10756023 -0.23356722  0.0621313  -0.00270992  0.00083944 -0.0026486 ]
Residuals (in sMPA frame) after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: 
      endpt_X  endpt_Y  endpt_Z
PR1   -0.021    0.059   -0.028
PR2    0.027   -0.074    0.008
PR3   -0.012    0.008    0.021
PR4   -0.025   -0.041   -0.014
PR5    0.030    0.048    0.012 

endpt_X    0.023704
endpt_Y    0.051174
endpt_Z    0.017934
dtype: float64


### 1.3 Convert new best-fit poses to encoder space

In [ ]:
df_after_fitting_FDPR_shifts_in_encoder_space = pd.DataFrame(columns=['X','Y','Z','Rx','Ry'],index=df_after_fitting_FDPR_shifts.index)
for pose in [val for val in pose_encoders.index if ('PR' in val) or ('PD' in val)]:
    df_after_fitting_FDPR_shifts_in_encoder_space.loc[pose] = calculate_encoders_from_5DOF(pd.DataFrame(df_after_fitting_FDPR_shifts.loc[pose]).T).squeeze()

print('New pose encoders, after incorporating the best-fit (x,y,z,Rx,Ry) shift to all poses: \n',df_after_fitting_FDPR_shifts_in_encoder_space,'\n')
print('Baseline pose encoders (displayed again here just for reference): \n', pose_encoders)

# 2 Write the new poses to an Excel file in the current directory

In [ ]:
write_to_excel = True

if write_to_excel == True:
    write_new_poses_to_Excel('test output (from FDPR update).xlsx','temporary plateau name',columns=columns,GSA_angle_WCS_deg=GSA_angle_WCS_deg,
                             df=df,df_encoders=pose_encoders,df_update=df_after_fitting_FDPR_shifts,df_update_encoders=df_after_fitting_FDPR_shifts_in_encoder_space,
                            focal_length=focal_length)